In [61]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import bs4
import time
import pandas as pd
import random

In [62]:
URL = r"https://docs.google.com/forms/d/1SadneeA-RnbGsx7uMkfV_iuE5v5MbAweDCOTPp0qVaE/viewform?edit_requested=true"

options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ['enable-automation'])
driver = webdriver.Chrome(options=options)

driver.get(URL)

time.sleep(1)
driver.execute_script("document.body.style.zoom='70%'")

In [63]:
def generate_random_answer():
    answers = [
        "ไหมไทย หัวใจสิน",
        "มนแคนต์ แก่นคูณ",
        "ลำไย ไห",
        "ติ๊ก ชีโร่"
    ]
    return random.choice(answers)


try:
    # ดึงข้อมูลหน้าเว็บ
    data = driver.page_source
    soup = bs4.BeautifulSoup(data, "html.parser")
    list_items = soup.select(
        '#mG61Hd > div.RH5hzf.RLS9Fe > div.lrKTG > div.o3Dpx > div')

    # วนลูปผ่านแต่ละ item
    for index, item in enumerate(list_items, 1):
        # 1. ตรวจสอบและจัดการกับ Radio options (ช้อยข้อสอบ)
        radio_options = driver.find_elements(By.CSS_SELECTOR,
                                             f"#mG61Hd > div.RH5hzf.RLS9Fe > div.lrKTG > div.o3Dpx > div:nth-child({index}) div[role='radio']")

        if radio_options:
            # สุ่มเลือกตัวเลือก (ยกเว้นตัวเลือกแรกถ้าเป็น "เลือก")
            valid_options = radio_options[1:] if len(
                radio_options) > 1 else radio_options
            if valid_options:
                random_option = random.choice(valid_options)
                option_text = random_option.text.strip()
                random_option.click()
                print(f"เลือกช้อยข้อสอบ: {option_text}")
                time.sleep(0.5)
                continue

        # 2. ตรวจสอบ Text input
        text_inputs = driver.find_elements(By.CSS_SELECTOR,
                                        f"#mG61Hd > div.RH5hzf.RLS9Fe > div.lrKTG > div.o3Dpx > div:nth-child({index}) input[type='text']")
        if text_inputs:
            answer = generate_random_answer()
            text_inputs[0].send_keys(answer)
            print(f"กรอกข้อความ: {answer}")
            continue

        # 3. ตรวจสอบ Dropdown
        dropdown = driver.find_elements(By.CSS_SELECTOR, 
            ".MocG8c.HZ3kWc.mhLiyf.LMgvRb.KKjvXb.DEh1R")

        if dropdown:
            # เลื่อน dropdown เข้ามาใน viewport
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", dropdown[0])
            time.sleep(0.5)  # ให้เวลา UI โหลด

            print(f"Dropdownมี: {len(dropdown)}ตัว")

            # คลิกที่ตัวเลือกแรก (เลือก)
            dropdown[0].click()
            time.sleep(1)

            # เลือกตัวเลือกอื่นใน dropdown
            options = driver.find_elements(By.CSS_SELECTOR, 
                ".MocG8c.HZ3kWc.mhLiyf.LMgvRb.KKjvXb.DEh1R[aria-selected='false']")

            if options:
                random_option = random.choice(options)
                random_option.click()
                print(f"เลือกตัวเลือกจาก dropdown: {random_option.text.strip()}")
                time.sleep(0.5)



        # 4. ตรวจสอบ Radio ในรูปแบบอื่น
        other_radio_options = driver.find_elements(By.CSS_SELECTOR,
                                                    f"#mG61Hd > div.RH5hzf.RLS9Fe > div.lrKTG > div.o3Dpx > div:nth-child({index}) div[role='option']")
        if other_radio_options:
            valid_options = [
                opt for opt in other_radio_options if opt.text.strip() != "เลือก"]
            if valid_options:
                random_option = random.choice(valid_options)
                option_text = random_option.text.strip()
                random_option.click()
                print(f"เลือกตัวเลือก radio: {option_text}")
                time.sleep(0.5)
            continue
    
    time.sleep(1)

    # ใช้ JavaScript เพื่อคลิกปุ่ม
    try:
        submit_button = driver.find_element(By.CSS_SELECTOR, "div[aria-label='Submit']")
        driver.execute_script("arguments[0].click();", submit_button)
        print("\n########### กดปุ่ม 'ส่ง' สำเร็จ ###########\n")
    except NoSuchElementException:
        print("ไม่พบปุ่มส่ง (Submit button) บนหน้าเว็บ")
        submit_button = driver.find_elements(By.CSS_SELECTOR, "div[role='button']")
        for idx, btn in enumerate(submit_button):
            print(f'ข้อความในButtonที่ {idx} role=button: {btn.text}')

            if "ถัดไป" in btn.text.strip():
                driver.execute_script("arguments[0].click();", btn)
                print("\n########### กดปุ่ม 'ถัดไป' สำเร็จ ###########\n")



except Exception as e:
    print(f"เกิดข้อผิดพลาด: {str(e)}")

กรอกข้อความ: ไหมไทย หัวใจสิน
กรอกข้อความ: ติ๊ก ชีโร่
Dropdownมี: 2ตัว
เลือกตัวเลือก radio: นาย
กรอกข้อความ: ติ๊ก ชีโร่
Dropdownมี: 1ตัว
เลือกตัวเลือก radio: ม.5/17
กรอกข้อความ: ไหมไทย หัวใจสิน
ไม่พบปุ่มส่ง (Submit button) บนหน้าเว็บ
ข้อความในButtonที่ 0 role=button: ขอสิทธิ์แก้ไข
ข้อความในButtonที่ 1 role=button: ถัดไป

########### กดปุ่ม 'ถัดไป' สำเร็จ ###########

เกิดข้อผิดพลาด: Message: stale element reference: stale element not found
  (Session info: chrome=132.0.6834.197); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6B18202F5+28725]
	(No symbol) [0x00007FF6B1782AE0]
	(No symbol) [0x00007FF6B161510A]
	(No symbol) [0x00007FF6B16298C4]
	(No symbol) [0x00007FF6B16283D1]
	(No symbol) [0x00007FF6B161CA89]
	(No symbol) [0x00007FF6B161A7E2]
	(No symbol) [0x00007FF6B161EABC]
	(No symbol) [0x00007FF6B161EB8F]
	(No symbol) [0x00007FF6B166313F]

In [64]:
try:
    # ดึงรายการ Radio Group ทั้งหมด
    radio_groups = driver.find_elements(By.CSS_SELECTOR, 'div[role="radiogroup"]')

    for idx, group in enumerate(radio_groups):
        print(f"\n🔹 รายการที่ {idx + 1}\n{'-'*30}")

        # หา radio options ในกลุ่มนั้น
        radio_options = group.find_elements(By.CSS_SELECTOR, 'div[role="radio"]')

        # กรองตัวเลือกที่ยังไม่ได้เลือก (aria-checked="false")
        unchecked_options = [opt for opt in radio_options if opt.get_attribute("aria-checked") == "false"]

        if unchecked_options:
            # สุ่มเลือก 1 ตัวจากตัวเลือกที่ยังไม่ถูกเลือก
            random_choice = random.choice(unchecked_options)
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", random_choice)
            time.sleep(0.5)  # หน่วงเวลาให้ UI โหลด

            # คลิกเลือกตัวเลือก
            random_choice.click()
            print(f"✅ เลือก: {random_choice.get_attribute('aria-label')}")

        else:
            print("⚠️ ไม่มีตัวเลือกที่ยังไม่ได้เลือก")

        time.sleep(1)  # พักก่อนเลือกกลุ่มถัดไป

except Exception as e:
    print(f"EXCEPT: {e}")




🔹 รายการที่ 1
------------------------------
✅ เลือก: ค. การช่วยเหลือผู้ที่หยุดหายใจ ให้มีการหายใจและการไหลเวียนเลือด เพื่อป้องกันไม่ให้ได้รับอันตราย

🔹 รายการที่ 2
------------------------------
✅ เลือก: ง. RCP

🔹 รายการที่ 3
------------------------------
✅ เลือก: ก. Cardiac

🔹 รายการที่ 4
------------------------------
✅ เลือก: ง. ให้ดื่มน้ำเพื่อปกกันการขาดน้ำ

🔹 รายการที่ 5
------------------------------
✅ เลือก: ข. 4 นาที

🔹 รายการที่ 6
------------------------------
✅ เลือก: ข. B-A-C

🔹 รายการที่ 7
------------------------------
✅ เลือก: ค. เหนือปลายกระดูกซี่โครงขั้นมา 2 นิ้วมือ

🔹 รายการที่ 8
------------------------------
✅ เลือก: ก. ผายปอดเป่าลมเข้าปาก

🔹 รายการที่ 9
------------------------------
✅ เลือก: ง. 1616

🔹 รายการที่ 10
------------------------------
✅ เลือก: ง. ขาดออกซิเจน


In [65]:
submit_button = driver.find_element(By.CSS_SELECTOR, "div[aria-label='Submit']")
driver.execute_script("arguments[0].click();", submit_button)
print("\n########### กดปุ่ม 'ส่ง' สำเร็จ ###########\n")



########### กดปุ่ม 'ส่ง' สำเร็จ ###########



In [66]:
view_score = driver.find_elements(By.CSS_SELECTOR, "div[role='presentation']")
for idx, btn in enumerate(view_score):
    text = btn.text.strip()
    print(f'ข้อความใน Button ที่ {idx}: {repr(text)}')

    if "ดูคะแนน" in text:
        print(f"\n🎯 พบปุ่มที่ตรงกับ 'ดูคะแนน' กำลังกด...\n")

        # เลื่อนให้ปุ่มอยู่ตรงกลางก่อนกด
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
        time.sleep(0.5)  # รอให้ UI โหลดก่อน

        try:
            btn.click()  # กดปกติ
            print("\n✅ กดปุ่ม 'ดูคะแนน' สำเร็จ (click)\n")
        except:
            driver.execute_script("arguments[0].click();", btn)
            print("\n✅ กดปุ่ม 'ดูคะแนน' สำเร็จ (JS click)\n")


ข้อความใน Button ที่ 0: 'ดูคะแนน'

🎯 พบปุ่มที่ตรงกับ 'ดูคะแนน' กำลังกด...


✅ กดปุ่ม 'ดูคะแนน' สำเร็จ (click)



In [67]:
time.sleep(2)
# ดึงรายการ window handles ทั้งหมด
tabs = driver.window_handles

# สลับไปยังแท็บใหม่ (แท็บล่าสุด)
driver.switch_to.window(tabs[-1])

print("✅ เปลี่ยนไปที่แท็บใหม่สำเร็จ!")


✅ เปลี่ยนไปที่แท็บใหม่สำเร็จ!


In [ ]:

driver.execute_script("document.body.style.zoom='70%'")
correct_label = driver.find_elements(By.CSS_SELECTOR, "label.docssharedWizToggleLabeledContainer.LygNqb.N2RpBe.O4MBef.RDPZE")

print(f"พบ {len(correct_label)} รายการที่ต้องตรวจสอบ\n")
i = 1
for idx, label in enumerate(correct_label):
    # ใช้ xpath เพื่อหา div ลูกตรงๆ ของ label
    div_children = label.find_elements(By.XPATH, "./div")
    
    # ตรวจสอบว่าใน div ลูกตรงๆ มี aria-label="ไม่ถูกต้อง"
    invalid_div = label.find_elements(By.XPATH, ".//div[@aria-label='ไม่ถูกต้อง']")
    
    # ข้ามกรณีที่พบ div ที่มี aria-label="ไม่ถูกต้อง"
    if invalid_div:
        # print(f"ข้อที่ {idx+1} ข้าม เพราะมี div ที่มี aria-label='ไม่ถูกต้อง'")
        continue

    print(f"\033[1;32mข้อที่ {i} ตอบ:\033[0m {label.text}")  # ใช้สีเขียวสำหรับข้อความที่ถูกต้อง
    
    i+=1
    print('-' * 50)  # แสดงเส้นแบ่งเพื่อให้อ่านง่ายขึ้น


พบ 19 รายการที่ต้องตรวจสอบ

ข้อที่ 1 ตอบ: ง. การช่วยเหลือผู้ที่หยุดหายใจหรือหัวใจหยุดเต้น ให้มีการหายใจและการไหลเวียนเลือดกลับคืนสู่สภาพปกติ
--------------------------------------------------
ข้อที่ 2 ตอบ: ข. CPR
--------------------------------------------------
ข้อที่ 3 ตอบ: ง. Respiratory Arrest
--------------------------------------------------
ข้อที่ 4 ตอบ: ก. ผายปอด
--------------------------------------------------
ข้อที่ 5 ตอบ: ข. 4 นาที
 
--------------------------------------------------
ข้อที่ 6 ตอบ: ค. C-A-B
--------------------------------------------------
ข้อที่ 7 ตอบ: ง. เหนือปลายกระดูกหน้าอกขั้นมา 2 นิ้วมือ
--------------------------------------------------
ข้อที่ 8 ตอบ: ข. เชยคาง ดันหน้าผาก
--------------------------------------------------
ข้อที่ 9 ตอบ: ค. 1669
--------------------------------------------------
ข้อที่ 10 ตอบ: ข. ซี่โครงหัก
--------------------------------------------------
